In [3]:
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/AD/
  inflating: data/AD/wwmAD_057_S_0474_64218.nii  
  inflating: data/AD/wwmAD_033_S_0889_83484.nii  
  inflating: data/AD/wwmAD_027_S_0404_82214.nii  
  inflating: data/AD/wwmAD_116_S_0392_86146.nii  
  inflating: data/AD/wwmAD_067_S_0029_119181.nii  
  inflating: data/AD/wwmAD_126_S_1221_91967.nii  
  inflating: data/AD/wwmAD_073_S_0565_119234.nii  
  inflating: data/AD/wwmAD_002_S_0938_118685.nii  
  inflating: data/AD/wwmAD_137_S_0796_91210.nii  
  inflating: data/AD/wwmAD_136_S_0299_119726.nii  
  inflating: data/AD/wwmAD_123_S_0091_119290.nii  
  inflating: data/AD/wwmAD_036_S_0577_72195.nii  
  inflating: data/AD/wwmAD_027_S_0850_94797.nii  
  inflating: data/AD/wwmAD_062_S_0793_82351.nii  
  inflating: data/AD/wwmAD_021_S_0753_78869.nii  
  inflating: data/AD/wwmAD_021_S_0343_85699.ni

In [4]:
import numpy as np
import random
import math
from PIL import Image
from skimage.transform import resize
import skimage
import torch
import matplotlib.pyplot as plt
class CustomResize(object):
    def __init__(self, network_type, trg_size):

        self.trg_size = trg_size
        self.network_type = network_type

    def __call__(self, img):
        resized_img = self.resize_image(img, self.trg_size)
        return resized_img

    def resize_image(self, img, trg_size):
        img_array = np.asarray(img.get_data())
        res = resize(img_array, trg_size, mode='reflect', anti_aliasing=False, preserve_range=True)

        # type check
        if type(res) != np.ndarray:
            raise "type error!"

        # PIL image cannot handle 3D image, only return ndarray type, which ToTensor accepts
        return res

class CustomToTensor(object):
    def __init__(self, network_type):

        self.network_type = network_type

    def __call__(self, pic):

        if isinstance(pic, np.ndarray):
            
            img = torch.from_numpy(pic.transpose((2, 0, 1)))
            
            # backward compatibility
            return img.float().div(255)

In [5]:
from torch.utils.data import Dataset
class AD_Standard_CNN_Dataset(Dataset):
    """labeled Faces in the Wild dataset."""
    
    def __init__(self, root_dir, data_file, transform=None, noise=True):
        """
        Args:
            root_dir (string): Directory of all the images.
            data_file (string): File name of the train/test split file.
            transform (callable, optional): Optional transform to be applied on a sample.
            data_augmentation (boolean): Optional data augmentation.
        """
        self.root_dir = root_dir
        self.data_file = data_file
        self.transform = transform
        self.noise = noise
    
    def __len__(self):

        return sum(1 for line in open(self.data_file))
    def __getitem__(self, idx):
        df = open(self.data_file)
        lines = df.readlines()
        lst = lines[idx].split()
        img_name = lst[0]
        img_label = lst[1]
        image_path = os.path.join(self.root_dir,img_label, img_name)
        image = nib.load(image_path)
        label=0
        if img_label == 'Normal':
            label = 0
        elif img_label == 'AD':
            label = 1
        elif img_label == 'MCI':
            label = 2
        
        image_array = np.array(image.get_data())
        if self.noise:
            image_array = gaussianNoise(image_array)
        image_array = customToTensor(image_array)
        sample = {'image': image_array, 'label': label}
        
        return sample

def customToTensor(pic):
    if isinstance(pic, np.ndarray):
        img = torch.from_numpy(pic)
        img = torch.unsqueeze(img,0)
        # backward compatibility
        return img.float()

def gaussianNoise(img_array):
    var_lst = [0, 0.0005, 0.00075, 0.001, 0.0025, 0.005]
    w,h,d= img_array.shape
    mean = 0
    var = random.choice(var_lst)
    sigma = var**0.5
    gauss_noise = np.random.normal(mean,sigma,(w,h,d))
    gauss_noise = gauss_noise.reshape(w,h,d)
    noise_image = img_array + gauss_noise
    return noise_image

In [6]:
import torch

import torch.nn as nn
import math

class CNN(nn.Module):
    def __init__(self, num_classes=3):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 410, kernel_size=7, stride=7, padding=3)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool3d(kernel_size=7,stride=7)
        # self.conv2 = nn.Conv3d(410, 200, kernel_size=3, stride=1, padding=1)
        # self.relu2 = nn.ReLU(inplace=True)
        # self.pool2 = nn.MaxPool3d(kernel_size=3, stride=3)
        # self.fc1 = nn.Linear(5*5*5*200, 800)
        self.dropout1 = nn.Dropout(0.5)
        #self.fc1 = nn.Linear(2*3*2*410, 80)
        self.fc1 = nn.Linear(2*410, 80)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(80, num_classes)
        self.softmax = nn.LogSoftmax()
        self.parameter_initialization()



    def forward(self, out):
        out = self.pool1(self.relu1(self.conv1(out)))
        out = self.dropout1(out)
        # out = self.pool2(self.relu2(self.conv2(out)))
        # out = out.view(-1,5*5*5*200)
        #print(out.shape)
        #out = out.view(-1, 2*3*2*410)
        out = out.view(-1, 2*410)

        out = self.fc1(out)
        out = self.dropout2(out)
        out = self.fc2(out)

        out = self.softmax(out)

        return out

    def parameter_initialization(self):
        stdv = 1.0 / math.sqrt(410)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)



In [8]:
import argparse
import logging
import os
import nibabel as nib
import torch
import torch.nn as nn
from torch import cuda
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
import warnings
warnings.filterwarnings('ignore')
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils
from PIL import Image

import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import random
logging.basicConfig(
    format='%(asctime)s %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO)

parser = argparse.ArgumentParser(description="Starter code for CNN .")

parser.add_argument("--epochs", default=20, type=int,
                    help="Epochs through the data. (default=20)")  
parser.add_argument("--learning_rate", "-lr", default=1e-3, type=float,
                    help="Learning rate of the optimization. (default=0.01)")
parser.add_argument('--weight_decay', '--wd', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)')         
parser.add_argument("--batch_size", default=2, type=int,
                    help="Batch size for training. (default=1)")
parser.add_argument("--gpuid", default=[0], nargs='+', type=int,
                    help="ID of gpu device to use. Empty implies cpu usage.")
parser.add_argument("--autoencoder", default=False, type=bool,
                    help="Whether to use the parameters from pretrained autoencoder.")
parser.add_argument("--num_classes", default=3, type=int,
                    help="The number of classes, 2 or 3.")
parser.add_argument("--estop", default=1e-5, type=float,
                    help="Early stopping criteria on the development set. (default=1e-2)")  
parser.add_argument("--noise", default=True, type=bool,
                    help="Whether to add gaussian noise to scans.")
# feel free to add more arguments as you need



def main(options):
    # Path configuration
    if options.num_classes == 2:
        TRAINING_PATH = 'train.txt'
        TESTING_PATH = 'validation_2C_new.txt'
    else:
        TRAINING_PATH = 'label.txt'
        TESTING_PATH = 'test.txt'
    IMG_PATH = './data'

    trg_size = (121, 145, 121)
    
    # transformations = transforms.Compose([CustomResize("CNN", trg_size),
    #                                       CustomToTensor("CNN")
    #                                     ])

    dset_train = AD_Standard_CNN_Dataset(IMG_PATH, TRAINING_PATH, noise=True)
    dset_test = AD_Standard_CNN_Dataset(IMG_PATH, TESTING_PATH, noise=False)

    # Use argument load to distinguish training and testing

    train_loader = DataLoader(dset_train,
                              batch_size = options.batch_size,
                              shuffle = True,
                              num_workers = 4,
                              drop_last = True
                              )

    test_loader = DataLoader(dset_test,
                             batch_size = options.batch_size,
                             shuffle = False,
                             num_workers = 4,
                             drop_last=True
                             )

    use_cuda = (len(options.gpuid) >= 1)
    # if options.gpuid:
    #     cuda.set_device(options.gpuid[0])

    # Training process
    model = CNN(options.num_classes)

    if use_cuda > 0:
        model = model.cuda()
    else:
        model.cpu()

    if options.autoencoder:
        pretrained_ae = torch.load("./autoencoder_pretrained_model39")
        model.state_dict()['conv1.weight'] = pretrained_ae['encoder.weight'].view(410,1,7,7,7)
        model.state_dict()['conv1.bias'] = pretrained_ae['encoder.bias']

        for p in model.conv1.parameters():
            p.requires_grad = False

    criterion = torch.nn.NLLLoss()

    lr = options.learning_rate
    optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, model.parameters()), lr, weight_decay=options.weight_decay)

    # main training loop
    last_dev_loss = 1e-4
    max_acc = 0
    max_epoch = 0
    f1 = open("cnn_autoencoder_loss_train.txt", 'a')
    f2 = open("cnn_autoencoder_loss_dev.txt", 'a')
    for epoch_i in range(options.epochs):
        logging.info("At {0}-th epoch.".format(epoch_i))
        train_loss = 0.0
        correct_cnt = 0.0

        for it, train_data in enumerate(train_loader):
            data_dic = train_data

            if use_cuda:
                imgs, labels = Variable(data_dic['image']).cuda(), Variable(data_dic['label']).cuda() 
            else:
                imgs, labels = Variable(data_dic['image']), Variable(data_dic['label'])

            # add channel dimension: (batch_size, D, H ,W) to (batch_size, 1, D, H ,W)
            # since 3D convolution requires 5D tensors
            img_input = imgs#.unsqueeze(1)
            #print(img_input.shape,labels.shape)
            integer_encoded = labels.data.cpu().numpy()

            # target should be LongTensor in loss function
            ground_truth = Variable(torch.from_numpy(integer_encoded)).long()
            if use_cuda:
                ground_truth = ground_truth.cuda()
                
            train_output = model(img_input)

            train_prob_predict = F.softmax(train_output, dim=1)
            _, predict = train_prob_predict.topk(1)
            #print(ground_truth.shape,predict.shape)
            loss = criterion(train_output, ground_truth)

            train_loss += loss
            correct_this_batch = (predict.squeeze(1) == ground_truth).sum().float()
            correct_cnt += correct_this_batch
            accuracy = float(correct_this_batch) / len(ground_truth)
            #logging.info("batch {0} training loss is : {1:.5f}".format(it, loss.data))
            #logging.info("batch {0} training accuracy is : {1:.5f}".format(it, accuracy))
            f1.write("batch {0} training loss is : {1:.5f}\n".format(it, loss.data))
            f1.write("batch {0} training accuracy is : {1:.5f}\n".format(it, loss.data))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_avg_loss = train_loss / (len(dset_train) / options.batch_size)
        train_avg_acu = float(correct_cnt) / len(dset_train)
        logging.info("Average training loss is {0:.5f} at the end of epoch {1}".format(train_avg_loss.data, epoch_i))
        logging.info("Average training accuracy is {0:.5f} at the end of epoch {1}".format(train_avg_acu, epoch_i))
        
        # validation -- this is a crude esitmation because there might be some paddings at the end
        dev_loss = 0.0
        correct_cnt = 0.0
        model.eval()
        for it, test_data in enumerate(test_loader):
            data_dic = test_data

            if use_cuda:
                imgs, labels = Variable(data_dic['image'], volatile=True).cuda(), Variable(data_dic['label'], volatile=True).cuda() 
            else:
                imgs, labels = Variable(data_dic['image'], volatile=True), Variable(data_dic['label'], volatile=True)

            img_input = imgs#.unsqueeze(1)
            integer_encoded = labels.data.cpu().numpy()
            ground_truth = Variable(torch.from_numpy(integer_encoded), volatile=True).long()
            if use_cuda:
                ground_truth = ground_truth.cuda()
            test_output = model(img_input)
            test_prob_predict = F.softmax(test_output, dim=1)
            _, predict = test_prob_predict.topk(1)
            loss = criterion(test_output, ground_truth)
            dev_loss += loss
            correct_this_batch = (predict.squeeze(1) == ground_truth).sum().float()
            correct_cnt += (predict.squeeze(1) == ground_truth).sum()
            accuracy = float(correct_this_batch) / len(ground_truth)
            #logging.info("batch {0} dev loss is : {1:.5f}".format(it, loss.data))
            #logging.info("batch {0} dev accuracy is : {1:.5f}".format(it, accuracy))
            f2.write("batch {0} dev loss is : {1:.5f}\n".format(it, loss.data))
            f2.write("batch {0} dev accuracy is : {1:.5f}\n".format(it, accuracy))

        dev_avg_loss = dev_loss / (len(dset_test) / options.batch_size)
        dev_avg_acu = float(correct_cnt) / len(dset_test)
        logging.info("Average validation loss is {0:.5f} at the end of epoch {1}".format(dev_avg_loss.data, epoch_i))
        logging.info("Average validation accuracy is {0:.5f} at the end of epoch {1}".format(dev_avg_acu, epoch_i))

        if dev_avg_acu > max_acc:
            max_acc = dev_avg_acu
            max_epoch = epoch_i

        #if (abs(dev_avg_loss.data[0] - last_dev_loss) <= options.estop) or ((epoch_i+1)%20==0):
        if max_acc>=0.75:
            torch.save(model.state_dict(), open("3DCNN_model_" + str(epoch_i) + '_' + str(max_acc), 'wb'))
        last_dev_loss = dev_avg_loss.data
        logging.info("Maximum accuracy on dev set is {0:.5f} for now".format(max_acc))
    logging.info("Maximum accuracy on dev set is {0:.5f} at the end of epoch {1}".format(max_acc, max_epoch))
    f1.close()
    f2.close()

if __name__ == "__main__":
  ret = parser.parse_known_args()
  options = ret[0]
  if ret[1]:
    logging.warning("unknown arguments: {0}".format(parser.parse_known_args()[1]))
  main(options)

2020-08-01 04:30:11 WARNING: unknown arguments: ['-f', '/root/.local/share/jupyter/runtime/kernel-6bbd7193-29c6-4103-9fd3-957359bc5ef9.json']
2020-08-01 04:30:11 INFO: At 0-th epoch.
2020-08-01 04:30:21 INFO: Average training loss is 1.15271 at the end of epoch 0
2020-08-01 04:30:21 INFO: Average training accuracy is 0.36911 at the end of epoch 0
2020-08-01 04:30:21 INFO: Average validation loss is 1.10624 at the end of epoch 0
2020-08-01 04:30:21 INFO: Average validation accuracy is 0.36000 at the end of epoch 0
2020-08-01 04:30:21 INFO: Maximum accuracy on dev set is 0.36000 for now
2020-08-01 04:30:21 INFO: At 1-th epoch.
2020-08-01 04:30:31 INFO: Average training loss is 1.09397 at the end of epoch 1
2020-08-01 04:30:31 INFO: Average training accuracy is 0.34555 at the end of epoch 1
2020-08-01 04:30:32 INFO: Average validation loss is 1.11000 at the end of epoch 1
2020-08-01 04:30:32 INFO: Average validation accuracy is 0.36000 at the end of epoch 1
2020-08-01 04:30:32 INFO: Maxim

In [ ]:
import os
import random
TRAINING_PATH_AD = 'data/AD'
TRAINING_PATH_MCI = 'data/MCI'
TRAINING_PATH_normal = 'data/normal'
train=[TRAINING_PATH_AD,TRAINING_PATH_MCI,TRAINING_PATH_normal]
f = open("label.txt",'a')
for train_data in train:


    for img_name in os.listdir(train_data):

        f.writelines([img_name,' ',train_data.split('/')[-1]])
        f.write('\n')
with open('label.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # get all row
    sum = 0
    list = []
    for line in lines:  # i-th row
            # find first space 
        list.append(line)


with open('test.txt', 'a', encoding='utf-8') as g:
    a = random.sample(list, 50)  # sample 50 row randomly
    for i in a:
        g.write(i)
    f.close()
    g.close()